### Toy Lindblad identification as POP solved with Moment SDP

In [1]:
using LinearAlgebra, JuMP, MosekTools
using Optim

Consider SDP optimization problem of the form:
\begin{align} 
&\operatorname{minimize}{ \left( \mathbf{trace}{(C X)} \right) }\\
&\operatorname{subject to} { y_i := \left( \mathbf{trace}{(A_i X)} \right) = b_i}, \quad i = 1... m, \quad
X \succcurlyeq 0
\end{align}

where $X\in\mathbf{S}^{n}$ is the decision variable, and each of the $A_{i}$ matrices and $C$ are also in $\mathbf{S}^{n}$  By the notation $\mathbf{S}^{n}$, we denote the set of all symmetric $n \times n$ matrices.

It could be solved with the following function:

In [2]:
function solve_SDP(A, b, C; quiet = false, term_tolerance = 1e-7)
    
    start_time = time()
    
    n = size(A)[1] # = size(C)[1]
    m = length(b)

    model = Model(optimizer_with_attributes(Mosek.Optimizer, "QUIET" => quiet, "INTPNT_CO_TOL_DFEAS" => term_tolerance))

    #set_silent(model)

    @variable(model, X[1:n, 1:n], PSD)

    @objective(model, Min, tr(C * X));
    
    @constraint(model, y[j=1:m], tr(A[:, (j - 1) * n + 1:j * n]* X) == b[j])

    optimize!(model)

    status = JuMP.termination_status(model)
    X_sol = JuMP.value.(X)
    obj_value = JuMP.objective_value(model)
    dual_sol = dual.(y)
    run_time = time() - start_time

    return model, status, run_time, X_sol, dual_sol, obj_value
    
end

solve_SDP (generic function with 1 method)

Consider simplified Lindblad type equation:

$
    \frac{d\rho}{dt} = - i[H, \rho]+\gamma\left[\sigma \rho \sigma^\dagger - \frac{1}{2}\left\{ \sigma^\dagger \sigma, \rho \right\} \right],
$

where
$\sigma = |0\rangle \langle 1|  = \begin{pmatrix} 0 & 1 \\ 0 & 0
   \end{pmatrix}$, 
   $\sigma^\dagger = \begin{pmatrix} 0 & 0 \\ 1 & 0
   \end{pmatrix}$,
   $\sigma^\dagger \sigma = \begin{pmatrix} 0 & 0 \\ 0 & 1
   \end{pmatrix}$ and
   $ H =  \begin{pmatrix} \omega & 0 \\ 0 & 0
   \end{pmatrix}$

To identify parameters of Lindblad equation we need to find optimum values of  angular frequency $\omega$ and decay rate $\gamma$.


We can formulate the following minimization problem using Pade approximation and just the initial and final state of the evolution (we do it here just for illustrative purposes as in practice we can do it only if $\Delta t$ is small):

$\min_{\gamma, H}{{\left\|{ \rho_f - \rho_0 - \Delta t {\mathcal{L}} \left[\frac{ \rho_f + \rho_0}{2} \right]} \right\|}_F^2}$

where  $\mathcal{L}$ is the Lindbladian operator:

$\mathcal{L} = - i[H, \rho]+\gamma\left[\sigma \rho \sigma^\dagger - \frac{1}{2}\left\{ \sigma^\dagger \sigma, \rho \right\} \right]$


If consider transfer:
$|y\rangle = \frac{|0\rangle + i|1\rangle}{\sqrt{2}} \rightarrow |\psi \rangle  = \frac{1}{\sqrt{3}}|0\rangle + \sqrt{\frac{2}{3}}|1\rangle$.

we have objective function:
$
L = {\left\|{ \rho_f - \rho_0 - \Delta t {\mathcal{L}} \left[\frac{ \rho_f + \rho_0}{2} \right]} \right\|}_F^2 =\frac{17}{12}\Delta t^2\omega^2 + 8 \sqrt2 \Delta t\omega + \frac{115}{48}\Delta t^2 \gamma^2 + \frac{7}{3}\Delta t\gamma + 24 
$,

where 

Introducing $x_1 = \Delta t \omega$ and $ x_2 = \Delta t \gamma$ we can rewrite the objective as a following polynomial optimization problem:

$
p(x) = 24 + 8 \sqrt2 x_1 + \frac{7}{3}x_2 + \frac{17}{12}x_1^2 + 0x_1x_2 + \frac{115}{48}x_2^2
$

$\min_{x \in \mathbb{R}^2 } p(x)$

In [3]:
function p(x)
    24 + 8√2x[1] + 7x[2]/3 + 17x[1]^2/12 + 115x[2]^2/48
end

p (generic function with 1 method)

For such a simple polynomial minimizers could be found analiticaly
or by standard numerical methods.

Analitical solution:

In [11]:
@show x₁ᵃⁿᵃˡⁱᵗ = -(48√2)/17
@show x₂ᵃⁿᵃˡⁱᵗ = -56/115
p([x₁ᵃⁿᵃˡⁱᵗ, x₂ᵃⁿᵃˡⁱᵗ])

x₁ᵃⁿᵃˡⁱᵗ = -(48 * √2) / 17 = -3.9930735878769745
x₂ᵃⁿᵃˡⁱᵗ = -56 / 115 = -0.48695652173913045


0.8436487638533702

Numerical solution with BFGS:

In [13]:
result = optimize(p, zeros(2), BFGS())
@show x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ = Optim.minimizer(result)
p([x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ])

(x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ) = Optim.minimizer(result) = [-3.9930735878943615, -0.48695652173788306]


0.8436487638533622

#### We can also try to solve our polynomial optimization problem as SDP in the space of moments

We can rewrite polynomial $p(x)$ as linear combination of monomials
$
p(x) = \sum_{\alpha \leq d} p_\alpha x^\alpha,   
$

where $x = \left( x_1, x_2 \right)^T$ and $\alpha = (\alpha_1, \alpha_2), \quad \alpha_i \in \{0 \dots d\}$, as in our case $x \in \mathbb{R}^2$ thus $d=2$ and for $\alpha$ we have all combinations of degrees $\alpha_i = 0,1,2$.

To solve our problem as SDP in the space of moments we move in the following steps:

#### 1. Infinite-dimensional LP
Reformulate the Nonlinear Optimization problem in terms of Probability Distributions (measures) to obtain the Infinite-dimensional Linear Programming problem in infinite-dimensional space of measures.

$\min_{\mu \in \mathcal{M}(\mathcal{X}} {\operatorname{E} [p(x)] = \sum_\alpha^\infty p_\alpha y_\alpha}$



#### 2. Infinite-dimensional SDP 

Reformulate the infinite-dimensional Problem of Step 1 in terms of Moments (higher order statistics) to obtain an infinite-dimensional Semi-Definite Programming problem.

$\min_{y \in \mathcal{Y}}{\operatorname{E} [p(x)] = \sum_\alpha^\infty p_\alpha y_\alpha}{}$
 
$\operatorname{s.t.}
 M_1(y), M_2(y), M_3(y) \dots \succcurlyeq 0$,
 
where $y_\alpha$ are moments defined as follows:
$
    y_\alpha := \int_\mathcal{X} x^\alpha \mu(dx),
$

#### 3.Finite SDP 

Truncate matrices of infinite-dimensional Semi-Definite Programming problem of Step 2 (use truncated moments) to obtain finite-dimensional Semi-Definite Programming problem.

For example, for two-dimensional $x \in \mathbb{R}^2 \mapsto p(x)$ we can write the first relaxation of it:
$\min_{y \in \mathbb{R}^6 } {\sum_{\alpha_1, \alpha_2 = 0}^{2} p_\alpha y_\alpha}$

$\operatorname{s.t.}{y_{00} = 1 }$

$  
 M_1(y) =\begin{pmatrix} 
y_{00} & y_{10} & y_{01}\\
y_{10} & y_{20} & y_{11}\\
y_{01} & y_{11} & y_{02} 
\end{pmatrix} \succcurlyeq 0
$

#### In our case:

Having polynomial optimization problem:

$
p(x) = 24 + 8 \sqrt2 x_1 + \frac{7}{3}x_2 + \frac{17}{12}x_1^2 + 0x_1x_2 + \frac{115}{48}x_2^2
$

$\min_{x \in \mathbb{R}^2 } p(x)$

we can write the first LMI relaxation for it:
    
$\min_{y \in \mathbb{R}^6 }
    24 y_{00} + 8 \sqrt2 y_{10} + \frac{7}{3} y_{01} + \frac{17}{12}y_{20} + 0y_{11} + \frac{115}{48}y_{02}$
    
$\operatorname{s.t.}{y_{00} = 1, M_1(y)\succcurlyeq 0}$

where

$M_1(y) =\begin{pmatrix} 
y_{00} & y_{10} & y_{01}\\
y_{10} & y_{20} & y_{11}\\
y_{01} & y_{11} & y_{02}
\end{pmatrix}$

This is SDP problem, 

$
\operatorname{minimize}{ \left( \mathbf{trace}{(C X)} \right) }\\
\operatorname{subject to} { y_i := \left( \mathbf{trace}{(A_i X)} \right) = b_i}, \quad i = 1... m, 
X \succcurlyeq 0
$

where:

$C =\begin{pmatrix} 
24 & 4 \sqrt2 & \frac{7}{6}\\
4 \sqrt2 & \frac{17}{12}  &0\\
\frac{7}{6} & 0 & \frac{115}{48}
\end{pmatrix}$

target variables:
$X =\begin{pmatrix} 
y_{00} & y_{10} & y_{01}\\
y_{10} & y_{20} & y_{11}\\
y_{01} & y_{11} & y_{02}
\end{pmatrix} \succcurlyeq 0$

and as $y_{00}=1$:

$A_1 =\begin{pmatrix} 
1 & 0 & 0\\
0 & 0 & 0\\
0 & 0 & 0
\end{pmatrix}, \quad b_1 = 1$


In [26]:
using DynamicPolynomials

@polyvar y[0:2, 0:2]

@polyvar x[1:2]

C = [24  4√2     7/6
     4√2 17/12  0     
     7/6 0       115/48]

Xˢʸᵐᵇ = [ y[1,1] y[2,1] y[1,2]
          y[2,1] y[3,1] y[2,3]
          y[1,2] y[2,2] y[1,3] ]

3×3 Matrix{PolyVar{true}}:
 y₀₋₀  y₁₋₀  y₀₋₁
 y₁₋₀  y₂₋₀  y₁₋₂
 y₀₋₁  y₁₋₁  y₀₋₂

We can check the coefficients are the same:

In [27]:
tr(C * Xˢʸᵐᵇ)

24.0y₀₋₀ + 11.313708498984761y₁₋₀ + 1.4166666666666667y₂₋₀ + 2.3333333333333335y₀₋₁ + 2.3958333333333335y₀₋₂

In [28]:
p(x)

1.4166666666666667x₁² + 2.3958333333333335x₂² + 11.313708498984761x₁ + 2.3333333333333335x₂ + 24.0

Additional constaint for $y_{00}$:

In [30]:
A = zeros(3,3)
A[1,1] = 1
b = 1

1

Now we can solve SDP

In [31]:
model, status, run_time, X_sol, dual_sol, obj_value = solve_SDP(A, b, C)
x₁ˢᵈᵖ, x₂ˢᵈᵖ = X_sol

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective se

3×3 Matrix{Float64}:
  1.0       -3.99308  -0.486957
 -3.99308   15.9447    1.94446
 -0.486957   1.94446   0.237127

#### Compare methods

In [33]:
x₁ˢᵈᵖ, x₂ˢᵈᵖ = X_sol[1,2], X_sol[1,3]

(-3.9930777191147366, -0.48695668992747426)

In [34]:
x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ

(-3.9930735878943615, -0.48695652173788306)

In [35]:
x₁ᵃⁿᵃˡⁱᵗ, x₂ᵃⁿᵃˡⁱᵗ

(-3.9930735878769745, -0.48695652173913045)

In [38]:
[x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ] - [x₁ᵃⁿᵃˡⁱᵗ, x₂ᵃⁿᵃˡⁱᵗ]

2-element Vector{Float64}:
 -1.738698074404965e-11
  1.2473910793175946e-12

The precission of SDP is around $10^{-6}$:

In [39]:
[x₁ˢᵈᵖ, x₂ˢᵈᵖ] - [x₁ᵃⁿᵃˡⁱᵗ, x₂ᵃⁿᵃˡⁱᵗ]

2-element Vector{Float64}:
 -4.13123776210611e-6
 -1.6818834380760705e-7